In [1]:
%gui qt
import napari
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from shapely.geometry import Polygon, Point

In [17]:
# SeqFISH data...


cellinfo = pd. read_csv("/Users/brian/Downloads/cellInfo.csv")

fov1 = pd. read_csv("/Users/brian/work/data/seqfish/cellxgene-fov1.csv", header = None)


In [33]:
fov1

,0,1,2,3,4,5,6,7,8,9,...,474,475,476,477,478,479,480,481,482,483
0,9330158h04rik,0,0,0,0,0,0,0,0,[7328.645961 4530.7206022 0.53105],...,0,0,0,0,0,0,0,0,0,0
1,A330009n23rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A830009l08rik,0,0,[7323.7706984 4465.6893372 0.2236;7322.4661042...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ackr2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Acsl5,0,0,[7321.4792456 4464.7590494 0.2236],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,Zfp703,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
575,Zfp804a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
576,Zfp804b,0,0,0,0,0,0,0,0,[7328.736519 4530.9609722 0.41925],...,0,0,0,0,0,0,0,0,0,0
577,Zfpm2,0,0,0,0,0,0,0,0,[7325.8994822 4529.8975306 0.7080294],...,0,0,0,0,0,0,0,0,0,0


In [26]:
cellinfo

,fov,cellID,vol_um,vol_px,centroid_um_1,centroid_um_2,centroid_px_1,centroid_px_2
0,0,1,9.796121e+07,27381824,10867.777769,7708.756969,3037.728580,2154.728580
1,0,2,0.000000e+00,0,NaN,NaN,NaN,NaN
2,0,3,0.000000e+00,0,NaN,NaN,NaN,NaN
3,0,4,0.000000e+00,0,NaN,NaN,NaN,NaN
4,0,5,0.000000e+00,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
7199,19,356,0.000000e+00,0,NaN,NaN,NaN,NaN
7200,19,357,1.598185e+05,44672,14104.267709,12923.659709,3942.382521,3612.382521
7201,19,358,0.000000e+00,0,NaN,NaN,NaN,NaN
7202,19,359,9.530726e+04,26640,14123.444078,12942.836078,3947.742643,3617.742643


In [25]:
np.sum(cellinfo["fov"]==0)

332

## Make a napari viewer...

In [3]:
v = napari.Viewer()

## Load cell x gene data from 1 dataset

In [42]:
all_cells = pd.read_csv("/Users/brian/Downloads/BARISTASEQ_cell_by_gene.csv")



### Add the cells as points, double-plotting cells with high Rorb, Cux2 if available

In [78]:
cells = v.add_points(all_cells[["x","y"]], symbol='o', name = "all", opacity = 0.8, blending = "translucent")
colorlist = ["magenta", "green","yellow","cyan", "blue", "orange"]
for ii, gene_to_plot in enumerate(["Cux2", "Atp1a3" , "Vxn", "Pcp4" , "Fezf2" , "Rorb" , "Gm11549"]):

    if gene_to_plot in all_cells:
        cux2condition = all_cells[gene_to_plot]>0.5*all_cells[gene_to_plot].median()
        cux2cells = v.add_points(all_cells[cux2condition][["x","y"]], symbol='disc',
                                 name = gene_to_plot+"+",edge_color= "black", size = 50,
                                 face_color= colorlist[ ii % len(colorlist)] , blending = "translucent", opacity = 0.3)


### annotate the layers in that viewer in a single `shapes` layer
#### Naming/order Convention:
0. outline of entire column as rectangle
1. Layer 1
1. Layer 2/3
2. Layer 4
3. Layer 5
4. Layer 6
5. single line segment, pointing from pia to white matter 

#### annotate the layer locations in a list

In [61]:
v.layers

[<Points layer 'all' at 0x1344a1350>,
 <Points layer 'Atp1a3+' at 0x131fcc310>,
 <Points layer 'Vxn+' at 0x13205a4d0>,
 <Points layer 'Pcp4+' at 0x132189bd0>,
 <Points layer 'Fezf2+' at 0x12141f850>,
 <Points layer 'Rorb+' at 0x132010290>,
 <Shapes layer 'Shapes' at 0x1330ff690>]

In [66]:
is_in_layer = {ii:[Polygon(p).intersects(Point(a))  for a  in v.layers[0].data] for ii,p in enumerate(v.layers[-1].data[:-2])}

In [ ]:
# ok:
# 1. make a couple functions to do this
#.  #. do a spatially partitioned search based on bounding box of polygon
# 2. throw annotation into separate vector matched to point cloud
#. 

In [67]:
is_in_layer

{0: [False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False

In [17]:
annotations = v.layers[4]

In [21]:
annotations.data

[array([[  322.69221525, 10409.33143621],
        [10023.76221747,  4247.2783876 ],
        [12605.16281892,  8285.92126406],
        [ 3112.27028456, 14864.32924839],
        [ 3112.27028456, 14864.32924839]]),
 array([[ 3195.5412717 , 14781.05826125],
        [  405.9632024 , 10492.60242336],
        [ 1696.66350312,  9659.89255192],
        [ 2529.37337456, 11533.48976265],
        [ 3445.35423313, 13074.00302481],
        [ 4194.79311743, 14073.25487053],
        [ 4194.79311743, 14073.25487053]]),
 array([[ 4278.06410457, 13948.34838981],
        [ 3903.34466242, 13573.62894767],
        [ 2820.82182956, 12033.11568551],
        [ 2196.28942598, 10784.05087836],
        [ 1655.02800955,  9618.25705835],
        [ 4028.25114314,  8161.01478334],
        [ 4611.14805314,  9493.35057764],
        [ 5776.94187315, 11491.85426908],
        [ 6526.38075744, 12366.19963409],
        [ 6526.38075744, 12366.19963409]]),
 array([[ 6568.01625101, 12366.19963409],
        [ 5860.21286029, 115

In [30]:
is_in_layer = {ii:[Polygon(p).intersects(Point(a))  for a  in v.layers[0].data] for ii,p in enumerate(annotations.data)}

In [68]:
v.add_points(all_cells[is_in_layer[1]][["x","y"]], symbol='o', size = 30)

<Points layer 'Points' at 0x133fe4390>

In [73]:
all_cells["layer"]  = np.zeros_like(all_cells["x"]) -1.

In [74]:
layer_keys=[None,"L1","L2/3","L4","L5","L6"]
for ii in is_in_layer.keys():
    all_cells["layer"][is_in_layer[ii]] = ii

/Users/brian/anaconda2/envs/starfish/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [77]:
np.sum(all_cells["layer"]==5)


1108

In [28]:
for ii,p in enumerate(annotations.data):
    print(p)

[[  322.69221525 10409.33143621]
 [10023.76221747  4247.2783876 ]
 [12605.16281892  8285.92126406]
 [ 3112.27028456 14864.32924839]
 [ 3112.27028456 14864.32924839]]
[[ 3195.5412717  14781.05826125]
 [  405.9632024  10492.60242336]
 [ 1696.66350312  9659.89255192]
 [ 2529.37337456 11533.48976265]
 [ 3445.35423313 13074.00302481]
 [ 4194.79311743 14073.25487053]
 [ 4194.79311743 14073.25487053]]
[[ 4278.06410457 13948.34838981]
 [ 3903.34466242 13573.62894767]
 [ 2820.82182956 12033.11568551]
 [ 2196.28942598 10784.05087836]
 [ 1655.02800955  9618.25705835]
 [ 4028.25114314  8161.01478334]
 [ 4611.14805314  9493.35057764]
 [ 5776.94187315 11491.85426908]
 [ 6526.38075744 12366.19963409]
 [ 6526.38075744 12366.19963409]]
[[ 6568.01625101 12366.19963409]
 [ 5860.21286029 11575.12525622]
 [ 4736.05453386  9659.89255192]
 [ 3944.98015599  8077.7437962 ]
 [ 5901.84835387  6911.94997619]
 [ 7941.98753888  9951.34100692]
 [ 8649.7909296  10992.22834622]
 [ 8649.7909296  10992.22834622]]
[[ 869

In [69]:
# depth coordinate calculation